In [4]:
import glob
import pyarrow
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
BASE = '../input/optiver-realized-volatility-prediction'

In [5]:
def ffill(data_df): 
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], np.arange(0,600)], names = ['time_id', 'seconds_in_bucket']), method=None)
    return data_df.reset_index()

In [ ]:
trade_files = glob.glob(f'{BASE}/trade_train.parquet/stock_id=*/*.parquet')

for file in tqdm(trade_files):
    trade_df = pd.read_parquet(file)
    ffill_df = ffill(trade_df)
    ffill_df["size"] = ffill_df["size"].fillna(0).astype(int)
    ffill_df["order_count"] = ffill_df["order_count"].fillna(0).astype(int)
    ffill_df['price'] = ffill_df.groupby('time_id')['price'].transform(lambda v: v.ffill())
    ffill_df['price'] = ffill_df.groupby('time_id')['price'].transform(lambda v: v.bfill())
    file_name = f'../input/trade_ffill/trade_train.parquet/' + file.split('/')[-2]
    save_df = pyarrow.Table.from_pandas(ffill_df)
    pyarrow.parquet.write_to_dataset(save_df, root_path=file_name)